## Import modules and relevant files

In [1]:
print 'importing modules  '
import os
import subprocess
from datetime import datetime as time
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import scipy.stats as stats


from Bio.Seq import Seq

%matplotlib inline




importing modules  


## Extracting universal sequences and barcodes from the reference files

## Creating a super contig

## Generate a FastQC report based on the fastq.gz file

Ran Fastqc using command line, everything looks fine. Note to myself: illumina gives the shitty reads first in the fastq file so it's ok if the first few 100s reads are mostly N. The fastqc report is on point. Now to align my barcodes!

HTML reports for both end reads in their respective folders

## Index sequences

In [2]:
yachie_lib_indexes = './yachie_lab_indexes.csv'

index_seq_dict = {}

with open(yachie_lib_indexes, 'r') as source:
    
    for line in source:
        
        line = line.strip('\n').strip('Index_')
        line = line.split(',')
        
        index_number = line[0]
        index_seq = line[1]
        
        index_seq_dict[index_number] = index_seq
       
        
print index_seq_dict

{'24': 'TAGTGGCCA', '20': 'TATCGTGCC', '21': 'TCCTGGTAT', '22': 'GGCAGAGGA', '23': 'ACAATGGAG', '1': 'CAAGTGTTC', '3': 'CACTAATGG', '2': 'AGGACATTC', '5': 'TTACGCTAA', '4': 'AGCCTGATG', '7': 'GTCGATGCA', '6': 'ACTCTCCGT', '9': 'CGCGCCCAG', '8': 'ACGGGAATT', '11': 'AGTATTACA', '10': 'ACTAGTTTG', '13': 'GTGAACCGA', '12': 'AGGTTGGGT', '15': 'CTGTCTTCG', '14': 'GCACAAAAC', '17': 'CAGCCCATA', '16': 'GACGCGACT', '19': 'AATACGCAC', '18': 'AGATATCTG'}


## Library index dispatch

In [4]:
index_to_library_mapping = './run_library_indexes_revised.csv'

lib_index_tuble_dict = {}
lib_index_by_tuple = {}


with open(index_to_library_mapping, 'r') as source:
    
    next(source)
    
    for line in source:
        
        line = line.strip('\n').strip('\r')
        line = line.split(',')
        
        lib_name = line[0]
        
        P1 = line[1]
        P2 = line[2]
        
        lib_tuple = tuple((P1,P2))
        
        lib_index_tuble_dict[lib_name] = lib_tuple
        lib_index_by_tuple[lib_tuple] = lib_name
        
print lib_index_tuble_dict
 



{'gal_16_B': ('16', '8'), 'gal_16_A': ('15', '8'), 'DSUR2_16_A': ('15', '14'), 'mock_16_A': ('15', '9'), 'mock_16_B': ('16', '9'), 'DSUR1_16_A': ('15', '12'), 'DSUR1_16_B': ('16', '12'), 'DS_16_A': ('15', '10'), 'DS_16_B': ('16', '10'), 'glu_16_A': ('15', '6'), 'glu_16_B': ('16', '6'), 'gly_16_A': ('15', '7'), 'DSUR2_16_B': ('16', '14'), 'gly_16_B': ('16', '7')}


# sequences for indexes and guide alignment

In [5]:
# amplicon
# NNNNN CAAGTGTTC TAACTTACGGAGTCGCTCTACG ATGTTTCGGCGTTCGAAACTTCTCCGCAGTGAAAGATAAATGATC NNNNNNNNNNNNNNNNNNNN GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGGTGCTTTTTTTGTTTTTTATGTCTTCGAGGCAAATTAAAGCCTTCGCAGGACCTAAAGAATCCCATCC GAACACTTG NNNNN
# degen index BC          PS1.0                            SNR52for                           guide                                    Sup4 term                                                                                                             P2 index   degen


for_adapter_end = 'GCTCTTCCGATCT'
# few nts before degen

degen_seq = 'AAAAA'
#bowtie1 cannot handle Ns, so I convert them to As

# P1 index seq

PS1_snr52for = 'TAACTTACGGAGTCGCTCTACGATGTTTCGGCGTTCGAAACTTCTCCGCAGTGAAAGATAAATGATC'

# guide

Sup4_termPS2 = 'GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGGTGCTTTTTTTGTTTTTTATGTCTTCGAGGCAAATTAAAGCCTTCGCAGGACCTAAAGAATCCCATCC'

# P2 index (RC)

# degen seq

rev_adapter_start = 'AGATCGGAAGAG'


guide_sequences = './chip_table_with_muts.tsv' 


all_by_all_fasta_file = './all_by_all_amplicon.fasta'

name = 'noDS_16_A'

     


# Fasta files for indexes

In [6]:
for_indexes = './for_indexes.fa'
rev_indexes = './rev_indexes.fa'


for_wseq = []

with open(for_indexes, 'w') as output:
    
    for library in lib_index_tuble_dict.keys():

        indexes = lib_index_tuble_dict[library]

        P1 = index_seq_dict[indexes[0]]
        
        if P1 not in for_wseq:
        
            fasta_header = '>'+ str(indexes[0]) + '\n'
            fasta_seq =  degen_seq + P1 + PS1_snr52for[:30] + '\n'

            output.write(fasta_header)
            output.write(fasta_seq)
            
            for_wseq.append(P1)

rev_wseq = []

with open(rev_indexes, 'w') as output:
    
    for library in lib_index_tuble_dict.keys():

        indexes = lib_index_tuble_dict[library]

        P2 = str(Seq(index_seq_dict[indexes[1]]).reverse_complement())
        
        if P2 not in rev_wseq:
        
            fasta_header = '>'+ str(indexes[1]) + '\n'
            fasta_seq = 'ACCTAAAGAATCCCATCC' + P2 + degen_seq + '\n'

            output.write(fasta_header)
            output.write(fasta_seq)
            
            rev_wseq.append(P2)
        
        
        

# Fasta file for guides

In [7]:
guides_fasta = './guides_only.fa'

with open(guides_fasta, 'w') as output:
    
    with open(guide_sequences, 'r') as source:

                next(source)

                for line in source:

                    line_data = line.split('\t')

                    guide_name = line_data[1]
                    guide_seq = line_data[6]
                    guide_target_type = line_data[2]

                    fasta_header = '>'+ guide_name + '\n'
                    fasta_seq = PS1_snr52for[30:] + guide_seq + Sup4_termPS2[0:30] +'\n'

                    output.write(fasta_header)
                    output.write(fasta_seq)

In [14]:
# command line calls (to be done in bash)

# for indexes
bowtie-build -f -r -o 4 for_indexes.fa hiseq_june2018_for_indexes

# rev indexes
bowtie-build -f -r -o 4 rev_indexes.fa hiseq_june2018_rev_indexes

# guides
bowtie-build -f -r -o 4 guides_only.fa hiseq_june2018_guides_only


SyntaxError: invalid syntax (<ipython-input-14-ae6044348ec0>, line 4)